<a href="https://colab.research.google.com/github/travisormsby/python-tips-tricks/blob/main/docs/GeospatialExamples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geospatial examples of Python Tips & Tricks
For the sake of efficiency, our workshop focused on toy examples that do not use GIS. Here are some examples that do. Note: These snippets have not been rigorously tested!

## Tip: Avoid a list where you could use a tuple

### 1a) Tuple-based storage

Scenario: You have CSVs stored in a subdirectory. Some of these CSVs can be converted to point shapefiles. For every CSV in the folder, you need to determine whether it contains a field called 'lat', which would indicate it has point coordinates.

In [ ]:
# Create list of file paths where the data is stored.
myDataPaths = [filePath for file in directory]


# Define a function for determining if a file meets your criteria.
def meetsCriteria(filePaths):
    """
    Dataframe must have a 'lat' field to be included.
    """
    members = []
    criterium = 'lat'

    for filePath in filePaths:
        with open(filePath) as fPath:
            headerList = csv.DictReader(fPath).fieldnames
            if criterium in headerList:
                members.append(filePath)
    return members


# Print all matching file paths
print(meetsCriteria(myDataPaths))

Action: Change the file paths collection to a tuple.

In [ ]:
# Exercise solution
myDataPaths = (filePath for file in directory)

## Tip: Use vectorized operations instead of updating line-by-line
We’ll look at how vectorized operations can improve script efficiency compared to line-by-line workflows like loops or cursors. We'll compare a loop using `pandas.DataFrame.apply()` to a vector-based workflow that relies on the geopandas built-in, `geopandas.GeoSeries.distance()`.

Vectorized operations—especially with GeoPandas or NumPy—are significantly more efficient than line-by-line processing. They leverage optimized C/NumPy backends, which makes them faster and more concise.

### Set up
Imagine you have a GeoDataFrame of points (e.g., city centers), and you want to compute the distance to a fixed location (e.g., a capital city or infrastructure hub).

In [ ]:
# Set up workspace
import geopandas as gpd
from shapely.geometry import Point

# Sample points: city centers
gdf = gpd.GeoDataFrame({
    'city': ['A', 'B', 'C'],
    'geometry': [Point(0, 0), Point(1, 1), Point(2, 2)]
}, crs='EPSG:4326')

# Fixed location (e.g., capital)
capital = Point(0, 1)

### Line-by-line using `.apply()`
With thousands of points, .apply() becomes dramatically slower because:
- It loops in Python (slow).
- It calls .distance() one-by-one (slow).
- No spatial index or batch processing (inefficient).

In [ ]:
# Line-by-line using apply
%%timeit
gdf['dist_to_capital'] = gdf['geometry'].apply(lambda x: x.distance(capital))

185 µs ± 17.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Vectorized operation using `.distance()`
Why this works:
- GeoPandas vectorized methods (e.g., .distance(), .within(), .intersects(), .buffer()) are backed by NumPy, Shapely, and GEOS—which means they use optimized C code.
- Much faster under the hood because GeoSeries.distance() is implemented in compiled code using spatial indexing and NumPy.
- You also avoid the overhead of Python loops and function calls.

In [ ]:
# Vectorized version
%%timeit
gdf['dist_to_capital_fast'] = gdf.distance(capital)

<magic-timeit>:1: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

<magic-timeit>:1: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



18.5 ms ± 544 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Tip: Use generators instead of functions to save memory
Has your script ever crashed from working on a single raster? Generators are often better when working on very large datasets, such as high resolution or multi-band rasters. Instead of loading the entire raster file into memory, a generator can help in chunking out your processing.

### Set up
Let’s look at nighttime lights raster data, like from the VIIRS or DMSP-OLS satellites. These rasters typically represent light intensity per pixel, which is great for analyzing urbanization, economic activity, or electricity access.

Say you want to compute the total light intensity or the mean light intensity over the whole raster. This can be done two ways: with a regular function or a generator-based approach.

In [ ]:
# Set up workspace
import rasterio
from rasterio.windows import Window

### Function
Reads the entire raster into memory.

Risk of crashing or memory exhaustion with large rasters (e.g., 30,000 x 30,000 px).

In [ ]:
# Load the raster and calculate the total light intensity
def read_all_and_sum(path):
    with rasterio.open(path) as src:
        data = src.read(1)  # Loads the entire band
        total = data.sum()
        count = data.size
        return total / count  # mean light intensity

### Generator
Efficient: Reads only a piece of the image at a time.

Scalable: Handles giant files with constant memory usage.

Composable: You can plug the generator into a pipeline—e.g., cloud masking, NDVI, etc. Easy to chain.

Better scalability of code: while you may develop your script first on a small area of interest, you may quickly want to replicate it to larger areas.

In [ ]:
def lights_block_generator(path, block_size=512):
    with rasterio.open(path) as src:
        for y in range(0, src.height, block_size):
            for x in range(0, src.width, block_size):
                w = min(block_size, src.width - x)
                h = min(block_size, src.height - y)
                window = Window(x, y, w, h)
                block = src.read(1, window=window)
                yield block

def mean_light_from_generator(path):
    total = 0
    count = 0
    for block in lights_block_generator(path):
        total += block.sum()
        count += block.size
    return total / count
